 # Comprehensive Exam

 ## Coding Artifact

 Kalin Gibbons

 Nov 20, 2020

 > Note: A hyperparameter is a numerical or other measurable factor
 responsible for some aspect of training a machine learning model, whose value
 cannot be estimated from the data, unlike regular parameters which represent
 inherent properties of the natural processes which generated data.

 ## Hyperparameter Optimization

 There are several python packages with automatic hyperparameter selection
 algorithms. A relatively recent contribution which I find particularly easy
 to use is [optuna](https://optuna.org/), which is detailed in this
 [2019 paper](https://arxiv.org/abs/1907.10902). Optuna allows the user to
 suggest ranges of values for parameters of various types, then utilizes a
 parameter sampling algorithms to find an optimal set of hyperparameters. Some
 of the sampling schemes available are:

 * Grid Search
 * Random
 * Bayesian
 * Evolutionary

While the parameter suggestion schemes available are:

 * Integers
   * Linear step
   * Logarithmic step
 * Floats
   * Logarithmic
   * Uniform
 * Categorical
   * List
   
 This notebook uses Optuna to implement hyperparameter tuning on a number of 
 ensemble algorithms.
 
 ## Imports

In [1]:
import os
import sys
import math
import logging
from pathlib import Path

from IPython.display import display, clear_output
from colorama import Fore, Style
import numpy as np
import scipy as sp
import scipy.io as spio
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor
)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import seaborn as sns
import pandas as pd

import optuna
from optuna.visualization import plot_optimization_history

import artifact
from artifact.datasets import load_tkr, tkr_group_lut
from artifact.helpers import RegressionProfile, REGRESSION_PROFILE_PATH



In [2]:

plt.rcParams['figure.figsize'] = (9, 5.5)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.size'] = 14
mpl.rcParams['font.family'] = 'Times New Roman'

# sns.set_context("poster")
# sns.set(rc={'figure.figsize': (16, 9.)})
# sns.set_style("whitegrid")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)


 Next, we'll select a functional group to examine, and only load the necessary
 data.
 ### Functional group selection

In [3]:
func_groups = list(tkr_group_lut.keys())
func_groups


['contact_mechanics', 'joint_loads', 'kinematics', 'ligaments', 'patella']

In [4]:
group = 'joint_loads'


 ### Loading the data

 We'll load a subset of the data containing the responses making up the chosen
 functional group.

In [5]:
shared_kwargs = dict(load_fcn=load_tkr, functional_group=group)
tkr_train = artifact.Results(**shared_kwargs, subset='train')
tkr_test = artifact.Results(**shared_kwargs, subset='test')
display(tkr_train.response_names[1:])

reg_prof = RegressionProfile(load_path=REGRESSION_PROFILE_PATH)
reg_prof.summarize(group)



['med_force_1',
 'med_force_2',
 'med_torque_1',
 'med_torque_2',
 'lat_force_1',
 'lat_force_2',
 'lat_torque_1',
 'lat_torque_2']



joint_loads
-----------

Best learners total by response:


LinearRegression                                                             4
AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)    2
GradientBoostingRegressor                                                    2
dtype: int64

med_torque_1    AdaBoostRegressor(base_estimator=DecisionTreeR...
lat_torque_1    AdaBoostRegressor(base_estimator=DecisionTreeR...
med_force_2                             GradientBoostingRegressor
lat_force_2                             GradientBoostingRegressor
med_force_1                                      LinearRegression
med_torque_2                                     LinearRegression
lat_force_1                                      LinearRegression
lat_torque_2                                     LinearRegression
dtype: object



Sorted by median RMS error (smallest to largest):


,count,mean,std,min,25%,50%,75%,max
"AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)",8.0,457.436336,615.357405,33.018228,62.568494,175.478336,557.412198,1486.255491
RandomForestRegressor,8.0,465.865525,624.637692,32.185607,65.538200,180.323926,567.866601,1514.090976
GradientBoostingRegressor,8.0,453.882096,605.045488,28.624252,59.692740,181.805110,561.551478,1471.967865
LinearRegression,8.0,447.428664,586.178902,33.169663,55.411237,185.267365,565.693107,1430.352219
"AdaBoostRegressor(base_estimator=LinearRegression, n_estimators=100)",8.0,537.220447,681.397757,36.384291,69.945561,234.843365,709.332590,1691.947807
DecisionTreeRegressor,8.0,669.671069,891.333514,47.775067,104.640293,254.695111,823.570689,2173.725325




RMS Errors:


,med_force_1,med_force_2,med_torque_1,med_torque_2,lat_force_1,lat_force_2,lat_torque_1,lat_torque_2
GradientBoostingRegressor,65.327128,28.624252,285.255098,1471.967865,78.355123,42.789574,293.734093,1365.003632
RandomForestRegressor,70.956263,32.185607,281.823194,1514.090976,78.824658,49.284010,285.853455,1413.906037
"AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)",67.577091,33.018228,277.754143,1486.255491,77.659439,47.542703,273.297233,1396.386362
"AdaBoostRegressor(base_estimator=LinearRegression, n_estimators=100)",72.443998,36.384291,390.646228,1691.947807,79.040501,62.450251,436.239931,1528.610569
DecisionTreeRegressor,116.544116,47.775067,426.638880,2173.725325,124.552855,68.928824,384.837367,2014.366115
LinearRegression,57.539003,33.169663,311.983333,1430.352219,67.756311,49.027940,302.778418,1326.822428


### Creating the optimization study

First we must define an objective function, which suggests the ranges of 
hyperparameters to be sampled. We can use switch-cases to optimize the machine 
learning algorithm itself, in addition to the hyperparameters.

In [6]:
learners = (
    # GradientBoostingRegressor(),
    # RandomForestRegressor(),
    # AdaBoostRegressor(DecisionTreeRegressor()),
    # AdaBoostRegressor(LinearRegression()),
    # DecisionTreeRegressor(),
    Ridge(),
    # AdaBoostRegressor()
)


def objective(trial, train, test, regressors):
    reg_strs = [r.__repr__() for r in regressors]
    regressor_name = trial.suggest_categorical('classifier', reg_strs)

    if regressor_name == 'GradientBoostingRegressor()':
        # learner_obj = GradientBoostingRegressor()
        pass

    elif regressor_name == 'RandomForestRegressor()':
        pass

    elif regressor_name == 'AdaBoostRegressor(base_estimator=DecisionTreeRegressor())':
        criterion = trial.suggest_categorical('criterion', [
            'mse', 'friedman_mse', 'mae', 'poisson'
        ])
        splitter = trial.suggest_categorical('splitter', ['best', 'random'])
        max_depth = trial.suggest_categorical('max_depth', [
             3, 4, 5
        ])
        min_samples_split = trial.suggest_categorical('min_samples_split', [
            2,
        ])
        min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0, 0.5)
        estimator = DecisionTreeRegressor(
            criterion=criterion,
            splitter=splitter,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf
        )

        loss = trial.suggest_categorical('loss', [
            'linear', 'square', 'exponential'
        ])
        n_estimators = trial.suggest_categorical('n_estimators', [100])
        learner_obj = AdaBoostRegressor(
            estimator,
            n_estimators=n_estimators,
            loss=loss
        )
        cv = 7

    elif regressor_name == 'AdaBoostRegressor(base_estimator=LinearRegression())':
        loss = trial.suggest_categorical('loss', [
            'linear', 'square', 'exponential'
        ])
        n_estimators = trial.suggest_categorical('n_estimators', [100])
        learner_obj = AdaBoostRegressor(
            LinearRegression(),
            n_estimators=n_estimators,
            loss=loss
        )
        cv = 7

    elif regressor_name == 'DecisionTreeRegressor()':
        criterion = trial.suggest_categorical('criterion', [
            'mse', 'friedman_mse', 'mae', 'poisson'
        ])
        splitter = trial.suggest_categorical('splitter', ['best', 'random'])
        max_depth = trial.suggest_categorical('max_depth', [
            3, 4, 5
        ])
        min_samples_split = trial.suggest_categorical('min_samples_split', [
            2,
        ])
        min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0, 0.5)
        learner_obj = DecisionTreeRegressor(
            criterion=criterion,
            splitter=splitter,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf
        )
        cv = 7

    elif regressor_name == 'Ridge()':
        # alpha = trial.suggest_loguniform('alpha', 1e-5, 10)
        alpha = trial.suggest_uniform('alpha', 4, 6)
        learner_obj = Ridge(alpha=alpha)
        cv = 7

    elif regressor_name == 'AdaBoostRegressorj()':
        pass

    else:
        pass

    regressor = artifact.Regressor(train,
                                   test,
                                   learner_obj,
                                   scaler=StandardScaler())
    scores = regressor.cross_val_score(n_jobs=-1, cv=cv)

    return scores.mean() * 100



### Running the optimization

Optuna will sample the parameters automatically, for a maximum number of trials 
specified.

In [7]:
study = optuna.create_study(direction='minimize')
study.optimize(
    lambda t: objective(t, tkr_train, tkr_test, learners),
    n_trials=50
)


[I 2021-02-15 17:07:16,032] A new study created in memory with name: no-name-4168baac-07b8-4805-a899-bcb5d43e7f53
[I 2021-02-15 17:07:21,271] Trial 0 finished with value: 4.2601779078720075 and parameters: {'classifier': 'Ridge()', 'alpha': 4.6455284209215435}. Best is trial 0 with value: 4.2601779078720075.
[I 2021-02-15 17:07:22,565] Trial 1 finished with value: 4.2597037770089985 and parameters: {'classifier': 'Ridge()', 'alpha': 5.0799562482928}. Best is trial 1 with value: 4.2597037770089985.
[I 2021-02-15 17:07:23,847] Trial 2 finished with value: 4.259750649652748 and parameters: {'classifier': 'Ridge()', 'alpha': 5.024862562128802}. Best is trial 1 with value: 4.2597037770089985.
[I 2021-02-15 17:07:25,145] Trial 3 finished with value: 4.261277954131018 and parameters: {'classifier': 'Ridge()', 'alpha': 4.026768429177228}. Best is trial 1 with value: 4.2597037770089985.
[I 2021-02-15 17:07:26,443] Trial 4 finished with value: 4.260064888516806 and parameters: {'classifier': 'Ri

In [13]:
plot_optimization_history(study).show()
print(study.best_trial)
print(Fore.YELLOW
      + f'\nBest trial\n  RMSE% = {study.best_value} \n  {study.best_params}')
print(Style.RESET_ALL)


FrozenTrial(number=19, value=4.259432792081609, datetime_start=datetime.datetime(2021, 2, 15, 17, 7, 51, 160070), datetime_complete=datetime.datetime(2021, 2, 15, 17, 7, 52, 749380), params={'classifier': 'Ridge()', 'alpha': 5.745238549741174}, distributions={'classifier': CategoricalDistribution(choices=('Ridge()',)), 'alpha': UniformDistribution(high=6, low=4)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=19, state=TrialState.COMPLETE)

Best trial
  RMSE% = 4.259432792081609 
  {'classifier': 'Ridge()', 'alpha': 5.745238549741174}



### Plotting the results from the optimization

We can assign the hyperparameters selected by optuna, and plot the resulting joint mechanics.

In [9]:
learner_strs = [lrn.__repr__() for lrn in learners]
learner_dict = dict(zip(learner_strs, learners))
learner_kwargs = study.best_params.copy()
learner = learner_dict[learner_kwargs['classifier']]
learner_kwargs.pop('classifier')
learner.set_params(**learner_kwargs)


Ridge(alpha=5.745238549741174)

In [10]:
lrn_name = type(learner).__name__
try:
    lrn_name = '-'.join((lrn_name, type(learner.base_estimator).__name__))
except AttributeError:
    pass

top_fig_dir = Path.cwd().parent / 'models' / 'predictions'
save_dir = top_fig_dir / group / lrn_name
n_rows, n_cols = 4, 3
tim = tkr_train.response['time'][0]
scaler = StandardScaler()
regr = artifact.Regressor(tkr_train, tkr_test, learner, scaler=scaler)
for resp_name in tkr_train.response_names:
    if resp_name == 'time':
        continue
    artifact.create_plots(n_rows, n_cols, regr, resp_name, save_dir)
    clear_output(wait=True)



In [11]:
view = artifact.plotting.ImageViewer(top_fig_dir)
view.show()
